In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000) #데이터 길이 확인을 위한

In [2]:
train = pd.read_csv('/content/drive/MyDrive/캐글스터디/SJU_challange_NLP/train.csv')
test = pd.read_csv('/content/drive/MyDrive/캐글스터디/SJU_challange_NLP/test.csv')

In [4]:
train['Text']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Great price and quick service, the product is a back case with no front protection, but alas thats why I ordered the sleeves for the front. Seems very durable and color is very

In [3]:
# null값 제거 
train.isnull().sum()
train_df=pd.DataFrame(train.dropna())
train_df.isnull().sum()

Unnamed: 0    0
Text          0
Label         0
dtype: int64

In [ ]:
train_df.lower()

In [4]:
# 데이터 토큰화
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
token = Tokenizer(num_words = 300, oov_token="<OOV>")
token.fit_on_texts(train_df['Text'])
word_index = token.word_index
print(word_index)
sequences_train = token.texts_to_sequences(train_df['Text'])
sequences_test = token.texts_to_sequences(test['Text'])
padded_train = pad_sequences(sequences_train,padding='post',maxlen=500)
padded_test = pad_sequences(sequences_test,padding='post',maxlen=500)


print(sequences_train)
print(sequences_test)

print(padded_train)
print(padded_test)

{'<OOV>': 1, 'the': 2, 'i': 3, 'it': 4, 'and': 5, 'to': 6, 'a': 7, 'is': 8, 'this': 9, 'my': 10, 'of': 11, 'for': 12, 'phone': 13, 'case': 14, 'on': 15, 'not': 16, 'in': 17, 'that': 18, 'with': 19, 'but': 20, 'was': 21, 'you': 22, 'have': 23, 'as': 24, 'so': 25, 'one': 26, 'screen': 27, 'very': 28, 'great': 29, 'like': 30, 'if': 31, 'be': 32, 'at': 33, 'good': 34, 'are': 35, 'all': 36, 'just': 37, 'or': 38, 'would': 39, 'product': 40, 'when': 41, 'from': 42, "it's": 43, 'had': 44, 'your': 45, 'will': 46, 'out': 47, 'they': 48, 'use': 49, 'no': 50, 'well': 51, 'get': 52, 'has': 53, 'up': 54, 'off': 55, 'me': 56, 'can': 57, 'an': 58, 'only': 59, 'protector': 60, 'after': 61, 'fit': 62, 'back': 63, 'charge': 64, 'does': 65, 'really': 66, 'time': 67, "don't": 68, 'battery': 69, 'iphone': 70, 'than': 71, 'work': 72, 'about': 73, 'because': 74, '2': 75, 'more': 76, 'do': 77, 'works': 78, 'there': 79, 'also': 80, 'even': 81, 'other': 82, 'nice': 83, 'too': 84, 'love': 85, 'what': 86, 'quality

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
range(len(sequences_train))

In [6]:
y_train = train['Label']

In [10]:
maxlen=0
length=0
for i in range(len(sequences_train)):
  length = len(sequences_train[i])
  if length>maxlen:
    maxlen = length

print(maxlen)

4457


In [ ]:
padded_train.shape

In [19]:
from keras.layers import *
from keras.models import *

model = Sequential()
model.add(Embedding(len(sequences_train),4))
model.add(Conv1D(256,5,activation='relu',input_shape=(None,500)))
model.add(Conv1D(128,5,activation='relu'))
model.add(Conv1D(64,5,activation='relu'))
model.add(GRU(32,activation='relu',dropout=0.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 4)           776556    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 256)         5376      
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 128)         163968    
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 64)          41024     
_________________________________________________________________
gru_3 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dense_9 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_10 (Dense)             (None, 32)               

In [22]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [18]:
#8차원 결과 // model.fit(padded_train,y_train,epochs=2,batch_size=64,validation_split=0.3)

Epoch 1/2
2124/2124 [==============================] - 2104s 990ms/step - loss: 0.6932 - acc: 0.4985 - val_loss: 0.6932 - val_acc: 0.5002
Epoch 2/2
2124/2124 [==============================] - 2114s 995ms/step - loss: 0.6931 - acc: 0.5039 - val_loss: 0.6932 - val_acc: 0.4998


In [23]:
model.fit(padded_train,y_train,epochs=2,batch_size=64,validation_split=0.3)

Epoch 1/2
  81/2124 [>.............................] - ETA: 32:44 - loss: 0.6935 - acc: 0.5108

KeyboardInterrupt: ignored

#다시 해보자 길이 줄여서

In [29]:
# 데이터 토큰화
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
token = Tokenizer(num_words = 300, oov_token="<OOV>")
token.fit_on_texts(train_df['Text'])
word_index = token.word_index
print(word_index)
sequences_train1 = token.texts_to_sequences(train_df['Text'])
sequences_test1 = token.texts_to_sequences(test['Text'])
padded_train1 = pad_sequences(sequences_train,padding='post',maxlen=50)
padded_test1 = pad_sequences(sequences_test,padding='post',maxlen=50)


print(sequences_train1)
print(sequences_test1)

print(padded_train1)
print(padded_test1)

{'<OOV>': 1, 'the': 2, 'i': 3, 'it': 4, 'and': 5, 'to': 6, 'a': 7, 'is': 8, 'this': 9, 'my': 10, 'of': 11, 'for': 12, 'phone': 13, 'case': 14, 'on': 15, 'not': 16, 'in': 17, 'that': 18, 'with': 19, 'but': 20, 'was': 21, 'you': 22, 'have': 23, 'as': 24, 'so': 25, 'one': 26, 'screen': 27, 'very': 28, 'great': 29, 'like': 30, 'if': 31, 'be': 32, 'at': 33, 'good': 34, 'are': 35, 'all': 36, 'just': 37, 'or': 38, 'would': 39, 'product': 40, 'when': 41, 'from': 42, "it's": 43, 'had': 44, 'your': 45, 'will': 46, 'out': 47, 'they': 48, 'use': 49, 'no': 50, 'well': 51, 'get': 52, 'has': 53, 'up': 54, 'off': 55, 'me': 56, 'can': 57, 'an': 58, 'only': 59, 'protector': 60, 'after': 61, 'fit': 62, 'back': 63, 'charge': 64, 'does': 65, 'really': 66, 'time': 67, "don't": 68, 'battery': 69, 'iphone': 70, 'than': 71, 'work': 72, 'about': 73, 'because': 74, '2': 75, 'more': 76, 'do': 77, 'works': 78, 'there': 79, 'also': 80, 'even': 81, 'other': 82, 'nice': 83, 'too': 84, 'love': 85, 'what': 86, 'quality

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
from keras.layers import *
from keras.models import *

model = Sequential()
model.add(Embedding(len(sequences_train),8))
model.add(Conv1D(128,5,activation='relu',input_shape=(None,40)))
model.add(Conv1D(64,5,activation='relu'))
model.add(Conv1D(32,5,activation='relu'))
model.add(GRU(32,activation='relu',dropout=0.1))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [43]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 8)           1553112   
_________________________________________________________________
conv1d_24 (Conv1D)           (None, None, 128)         5248      
_________________________________________________________________
conv1d_25 (Conv1D)           (None, None, 64)          41024     
_________________________________________________________________
conv1d_26 (Conv1D)           (None, None, 32)          10272     
_________________________________________________________________
gru_8 (GRU)                  (None, 32)                6336      
_________________________________________________________________
dense_24 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_25 (Dense)             (None, 32)               

In [44]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [45]:
model.fit(padded_train1,y_train,epochs=10,batch_size=64,validation_split=0.3)

Epoch 1/10
2124/2124 [==============================] - 219s 102ms/step - loss: 0.6764 - acc: 0.5590 - val_loss: 0.7454 - val_acc: 0.4987
Epoch 2/10
2124/2124 [==============================] - 215s 101ms/step - loss: 0.6323 - acc: 0.6581 - val_loss: 0.7657 - val_acc: 0.5000
Epoch 3/10
2124/2124 [==============================] - 213s 100ms/step - loss: 0.6277 - acc: 0.6636 - val_loss: 0.7576 - val_acc: 0.4999
Epoch 4/10
2124/2124 [==============================] - 208s 98ms/step - loss: 0.6265 - acc: 0.6654 - val_loss: 0.7832 - val_acc: 0.4994
Epoch 5/10
2124/2124 [==============================] - 211s 99ms/step - loss: 0.6216 - acc: 0.6725 - val_loss: 0.7728 - val_acc: 0.4988
Epoch 6/10
2124/2124 [==============================] - 215s 101ms/step - loss: 0.6155 - acc: 0.6805 - val_loss: 0.7749 - val_acc: 0.5008
Epoch 7/10
2124/2124 [==============================] - 212s 100ms/step - loss: 0.6172 - acc: 0.6789 - val_loss: 0.7634 - val_acc: 0.5000
Epoch 8/10
2124/2124 [==============